# Check and import lirabries/packages


**1. Checking packpage/libraries version**

In [4]:
import sys 
import tensorflow

print(f"Python version: {sys.version}")
print(f"Tensorflow version: {tensorflow.__version__}")

Python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Tensorflow version: 2.4.1


**2. Import libraries**

In [5]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer

# Get data from Google Drive

**3. Mount Drive**

In [1]:
from google.colab import drive, files
drive.mount('drive',force_remount=True)

Mounted at drive


In [ ]:
# checking if Drive is mounted and accessible
# If shared folder does not appear by using the mount command, then reset Colab runtime
# and mount using Web GUI (or vice versa)
%cd /content/drive/MyDrive/Dataset/
%ls

**4. Setup PATHS & VARIABLES**

In [102]:
# Define Path to the Dataset folder
BASE_PATH = '/content/drive/MyDrive/Dataset/MP4_OUTPUT'
VIDEOS_PATH = os.path.join(BASE_PATH,'**','*.mp4')
print(VIDEOS_PATH)

# Define LSTM sequence length and batch_size
SEQUENCE_LENGTH = 40
BATCH_SIZE = 16

/content/drive/MyDrive/Dataset/MP4_OUTPUT/**/*.mp4


**5. Sample Video**

We will not process every frame, but taking Kth sample where = num_frame_in_videos / SEQUENCE_LENGTH

In [103]:
# taking Kth sameple function
def frame_generator():
    video_paths = tf.io.gfile.glob(VIDEOS_PATH)
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0
        
        label = os.path.basename(os.path.dirname(video_path))
        
        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break
            
            # OpenCV reads in videos in BGR format so we need to rearrange the channels
            # to be in RGB format, resize the image, and preprocess it for the CNN
            if current_frame % sample_every_frame == 0:
                frame = frame[:, :, ::-1]
                img = tf.image.resize(frame, (224,224))
                img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
                max_images -= 1
                yield img, video_path
                
            if max_images == 0:
                break
            current_frame += 1

In [104]:
# Load Dataset
dataset = tf.data.Dataset.from_generator(frame_generator,
                                         output_types=(tf.float32,tf.string),
                                         output_shapes=((224,224,3),()))

# set batch_size 
dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [105]:
# Print shape and type of dataset
print(dataset)

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.string)>


# Build Feature Extraction Model

**6. Build Feature Extraction Model**

In [106]:
mobilenet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3),
                                                              include_top=False,
                                                              weights='imagenet')
x = mobilenet_v2.output

# Average Pooling - transforming the feature map from 8*8*2048 to 1x2048
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(x)
feature_extraction_model = tf.keras.Model(mobilenet_v2.input,pooling_output)

In [ ]:
# ONLY RUN ONCE
# Extract Feature - Generate .npy file for each video

current_path = None
all_features = []

# cycle through each img and extracts its features
# tqdm is a progress bar which updates each time the feature_extraction_model is called 

for img, batch_path in tqdm.tqdm(dataset):
  batch_features = feature_extraction_model(img)
  # Reshape tensor
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1))
  
  for features, path in zip(batch_features.numpy(), batch_path.numpy()):
    if path != current_path and current_path is not None:
      output_path = current_path.decode().replace('.mp4','.npy')
      np.save(output_path,all_features)
      all_features = []

    current_path = path
    all_features.append(features)

**7. Create Labels of our classes**

In [185]:
# load LABELS 
LABELS = ['good','heels_off','bent_over','knees_forward','knees_in','shallow']
# Encode to 0- not belong to class Or 1- belong to class using LabelBinarizer()
encoder = LabelBinarizer()
encoder.fit(LABELS )

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [186]:
# Checking output of LabelBinarizer()
print(encoder.classes_)
print(encoder.transform(LABELS))
print(encoder.inverse_transform(t))

['bent_over' 'good' 'heels_off' 'knees_forward' 'knees_in' 'shallow']
[[0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
['good' 'heels_off' 'bent_over' 'knees_forward' 'knees_in' 'shallow']


**8. LSTM Model**

Define LSTM Model with following layers:


*   Layer 1 = Masking Layer ( see keras [Doc](https://keras.io/api/layers/core_layers/masking/) )
*   Layer 2 = Defind what **1** cell of LSTM looks like [LSTM layer](https://keras.io/api/layers/recurrent_layers/lstm/). The total number of cells is defined as SEQUENCE_LENGTH above
*   Layer 3 = FNC (fully-connected layer) relu activation ( see [Dense layer](https://keras.io/api/layers/core_layers/dense/) )
*   Layer 4 = Drouput layer
*   Layer 5 = final decision FNC layer with softmax activation -- output has length of classes 



In [233]:
model = tf.keras.Sequential([
                            tf.keras.layers.Masking(mask_value=0.),
                             tf.keras.layers.LSTM(512,dropout=0.5,recurrent_dropout=0.5), # 512 units (hidden-layer)
                             tf.keras.layers.Dense(256,activation='relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(6,activation='softmax')
])
print(len(LABELS))

6


In [234]:
# Setup Loss function and metrics (see more at https://keras.io/api/metrics/)
model.compile(loss='categorical_crossentropy', # since we want to classify different type of squats
             optimizer='adam',
             metrics=['accuracy','top_k_categorical_accuracy']) # focus on accuracy

# Split Training/Test Data


**9. Splitting Training/Test data**

In [235]:
data_path=[]
for root,dir,filename in os.walk(BASE_PATH,topdown=True):
  for video_path in filename:
    if video_path.endswith('.mp4'):
      data_path.append(os.path.join(root,video_path))

# Shuffle video_paths and split to training-test(80-20)
import random     
print(len(data_path))
random.shuffle(data_path)
train_list = data_path[int((len(data_path)+1)*.20):] # get 80% of data to training
test_list = data_path[:int((len(data_path)+1)*.20)] # the rest go to testing

# number of train
print(f'Training: {len(train_list)}')
# number of test 
print(f'Test/Valid: {len(test_list)}')



289
Training: 231
Test/Valid: 58


In [236]:
# define make_generator() that returns a generator which will randomly shuffle the video list
# then building out the list as the .npy feature files

def make_generator(file_list):
  def generator():
    np.random.shuffle(file_list)
    for path in file_list:
      full_path = path.replace('.mp4','.npy')
      label = os.path.basename(os.path.dirname(path))
      features = np.load(full_path)

      padded_sequence = np.zeros((SEQUENCE_LENGTH,1280)) # MobileNet feature extractor 
      padded_sequence[0:len(features)] = np.array(features)

      transform_label = encoder.transform([label])
      yield padded_sequence, transform_label[0]
  return generator

In [ ]:
# setting Training/Test(Validation) Data
# Since we would use a new/unseen dataset for testing, thus the testing set here is used for validation

train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                                               output_types=(tf.float32,tf.int16),
                                               output_shapes=((SEQUENCE_LENGTH,1280),(len(LABELS))))
train_dataset = train_dataset.batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                                              output_types=(tf.float32,tf.int16),
                                              output_shapes=((SEQUENCE_LENGTH,1280),(len(LABELS))))

valid_dataset = valid_dataset.batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

# Print Train and Valid dataset
print(train_dataset)
print(valid_dataset)


# Train LSTM Model

**10. Train LSTM**

In [238]:
# Create Log_dir for TensorBoard Visualization
ROOT_PATH = '/content/drive/MyDrive/Dataset'
LOG_DIR = os.path.join(ROOT_PATH,'training_log')

# Create new dir if not exists
if not os.path.isdir(LOG_DIR): 
  !mkdir training_log


In [ ]:
# Callback function that will store information (checkpoints,etc) used for TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR,update_freq=1000)
model.fit(train_dataset,epochs=10,callbacks=[tensorboard_callback],verbose=2,validation_data=valid_dataset)

In [241]:
# Save model
model.file = os.path.join(BASE_PATH,'feature_extraction.h5')
model.save(model.file)

# Tensorboard and Evaluation on test_dataset

**11. Tensorboard Visualization**

In [ ]:
# Load the tensorboard notebook
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

**12. Run Evaluation**

In [252]:
print('----------------')
print('Evaluate on test Data')

# define test_set path 
TEST_PATH = ''
test_result = model.evaluate(test_dataset, verbose=1)
print(f'Test Loss, Test Accuracy: {test_result}')

----------------
Evaluate on test Data
3/3 [==============================] - 1s 267ms/step - loss: 1.4319 - accuracy: 0.5208 - top_k_categorical_accuracy: 1.0000
Test Loss, Test Accuracy: [1.4319251775741577, 0.5208333134651184, 1.0]


**13. Run Prediction**

In [ ]:
prediction = model.predict(valid_dataset,verbose=1)

print(prediction.shape)
print(prediction)